In [2]:
import subprocess
import platform
import re
import shutil

In [3]:
def get_available_wifi_networks():
    """
    Scan for available WiFi networks on Linux, Windows, and macOS.
    Returns a list of unique SSIDs.
    """
    networks = []
    system = platform.system()
    print(f"Scanning for WiFi networks on {system}...")
    try:
        if system == "Linux":
            # Use iwlist on Linux (this may require sudo privileges).
            result = subprocess.run(
                ["sudo", "iwlist", "wlan0", "scan"],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            for line in result.stdout.splitlines():
                line = line.strip()
                if line.startswith("ESSID:"):
                    ssid = line.split("ESSID:")[1].strip().strip('"')
                    if ssid and ssid not in networks:
                        networks.append(ssid)

        elif system == "Windows":
            # Use netsh command on Windows.
            result = subprocess.run(
                ["netsh", "wlan", "show", "networks", "mode=bssid"],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                shell=True
            )
            for line in result.stdout.splitlines():
                line = line.strip()
                # Windows output typically has lines like "SSID 1 : NetworkName"
                if line.startswith("SSID ") and ":" in line:
                    parts = line.split(":", 1)
                    ssid = parts[1].strip()
                    if ssid and ssid not in networks:
                        networks.append(ssid)

        elif system == "Darwin":  # macOS
            # First, try wdutil if available (airport is deprecated).
            wdutil_path = shutil.which("wdutil")
            print(f"wdutil_path: {wdutil_path}")
            if wdutil_path:
                result = subprocess.run(
                    [wdutil_path, "scan"],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True
                )
                print(f"result: {result}")
                # Assume output lines contain "SSID:"; parse accordingly.
                for line in result.stdout.splitlines():
                    line = line.strip()
                    if "SSID:" in line:
                        parts = line.split("SSID:")
                        if len(parts) > 1:
                            ssid = parts[1].strip()
                            if ssid and ssid not in networks:
                                networks.append(ssid)
            else:
                # Fallback to the deprecated airport command.
                airport_path = "/System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport"
                result = subprocess.run(
                    [airport_path, "-s"],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True
                )
                lines = result.stdout.splitlines()
                for line in lines:
                    # Skip any warning lines.
                    if line.startswith("WARNING"):
                        continue
                    line = line.strip()
                    if line:
                        # Split by multiple spaces to isolate the SSID (first column).
                        parts = re.split(r'\s{2,}', line)
                        if parts:
                            ssid = parts[0].strip()
                            if ssid and ssid not in networks:
                                networks.append(ssid)
        else:
            print(f"Unsupported operating system: {system}")
    except Exception as e:
        print(f"Error scanning WiFi networks on {system}: {e}")

    return networks

In [4]:
get_available_wifi_networks()

Scanning for WiFi networks on Darwin...
wdutil_path: /usr/bin/wdutil
result: CompletedProcess(args=['/usr/bin/wdutil', 'scan'], returncode=0, stdout='usage: sudo wdutil diagnose [-q] [-f outputDirectoryPath]\n            -q may be specified to suppress legal prompt and Finder window\n       sudo wdutil info\n       sudo wdutil log [{+|-} {system|wifi}]+\n       sudo wdutil dump\n       sudo wdutil clean\n       sudo wdutil privateMAC={0/1}\n', stderr='')


[]